# Modelo 2030 - Conversión de script a Notebook
Este notebook contiene la conversión del script `scene_2030.py` a celdas Jupyter organizadas: imports, carga de datos, parámetros, definición del modelo Pyomo, resolución y resultados.

Asegúrate de tener instaladas las dependencias: `pandas`, `pyomo`, y el solver `HiGHS` (o ajustar el solver si no está disponible).

## 1. Importar Librerías y Configuración Inicial

In [41]:
# Imports
import pandas as pd
import highspy as hgs
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Parámetros de configuración
tolerancia = 0.00001
perdida = 0.04

## 2. Cargar y Procesar Datos de Centrales Eléctricas

In [42]:
# Cargar archivos CSV (asegúrate que los archivos existan en el mismo directorio)
df_centrales_ex = pd.read_csv('centrales_ex_ED_CS.csv')
df_centrales_ex = df_centrales_ex.fillna(0)

df_centrales_nuevas = pd.read_csv('centrales_n_ED_CS.csv')
df_centrales_nuevas = df_centrales_nuevas.fillna(0)

# Preparar parámetros (index por nombre de planta)
param_centrales = df_centrales_ex.set_index(['planta_n'])
param_centrales_nuevas = df_centrales_nuevas.set_index(['planta_n'])


normas_emision = {'carbon': {'MP': 0.99 , 'SOx' : 0.95, 'NOx' : 0.0},
               'cc-gnl': {'MP': 0.95 , 'SOx' : 0.0, 'NOx' : 0.9}, 
               'petroleo_diesel': {'MP': 0.95 , 'SOx' : 0.0, 'NOx' : 0.0}}
# Revisar las primeras filas
#display(df_centrales_ex.head())
#display(df_centrales_nuevas.head())

In [94]:
# Cargar el DataFrame
df_combos = pd.read_csv('combos.csv')

# Reemplazar NaN con None
df_combos = df_combos.where(pd.notnull(df_combos), None)

# Convertir el DataFrame a una lista de listas
lista_combos = df_combos.values.tolist()

dic_combos_ex = {}
for planta in param_centrales.index.to_list():
    dic_combos_ex[planta] = lista_combos

dic_combos_nuevas = {}
for planta in param_centrales_nuevas.index.to_list():
    dic_combos_nuevas[planta] = lista_combos

index_combos = []
for i in range(1, 65):
    index_combos.append(f'combo_{i}')

print(index_combos)

['combo_1', 'combo_2', 'combo_3', 'combo_4', 'combo_5', 'combo_6', 'combo_7', 'combo_8', 'combo_9', 'combo_10', 'combo_11', 'combo_12', 'combo_13', 'combo_14', 'combo_15', 'combo_16', 'combo_17', 'combo_18', 'combo_19', 'combo_20', 'combo_21', 'combo_22', 'combo_23', 'combo_24', 'combo_25', 'combo_26', 'combo_27', 'combo_28', 'combo_29', 'combo_30', 'combo_31', 'combo_32', 'combo_33', 'combo_34', 'combo_35', 'combo_36', 'combo_37', 'combo_38', 'combo_39', 'combo_40', 'combo_41', 'combo_42', 'combo_43', 'combo_44', 'combo_45', 'combo_46', 'combo_47', 'combo_48', 'combo_49', 'combo_50', 'combo_51', 'combo_52', 'combo_53', 'combo_54', 'combo_55', 'combo_56', 'combo_57', 'combo_58', 'combo_59', 'combo_60', 'combo_61', 'combo_62', 'combo_63', 'combo_64']


In [ ]:
# Equipos de abatimiento
df_equipo_mp = pd.read_csv('abatimiento/equipo_mp.csv')
df_equipo_mp = df_equipo_mp.fillna(0)
equipo_mp = df_equipo_mp.set_index(['Equipo_MP'])

df_equipo_nox = pd.read_csv('abatimiento/equipo_nox.csv')
df_equipo_nox = df_equipo_nox.fillna(0)
equipo_nox = df_equipo_nox.set_index(['Equipo_NOx'])

df_equipo_sox = pd.read_csv('abatimiento/equipo_sox.csv')
df_equipo_sox = df_equipo_sox.fillna(0)
equipo_sox = df_equipo_sox.set_index(['Equipo_SOx'])

# Daño ambiental (costo social)

df_dano_ambiental = pd.read_csv('dano_ambiental.csv')
df_dano_ambiental = df_dano_ambiental.fillna(0)
dano_ambiental = df_dano_ambiental.set_index(['Ubicacion'])

display(equipo_mp.head())
display(equipo_nox.head())
display(equipo_sox.head())
display(dano_ambiental.head())

## 4. Definir Parámetros del Sistema y Constantes

In [ ]:
# Tipos de Centrales
t_centrales = ['biomasa', 'carbon','cc-gnl', 'petroleo_diesel', 'hidro', 'minihidro','eolica','solar', 'geotermia']
t_ernc = ['eolica','solar', 'geotermia','minihidro','hidro' ]
dispnibilidad_hidro = [0.8215,0.6297,0.561]
costo_falla = 505.5
year = 2030 - 2016

# Constantes para pasar de GWh a Toneladas de combustible (calculado en el excel)
conversion_GWh_a_ton = {'petroleo_diesel': 79.6808152,
                         'cc-gnl' : 61.96031117,
                            'carbon': 132.3056959}

## Se supone que si cada uno de estos factores lo multiplicamos GWh * factor * emisiones descontroladas
## nos da las emisiones en kg de cada contaminante (MP, SOx, NOx, CO2) 
## luego hay que pasarlo a toneladas dividiendo por 1000
## y finalmente para pasarlo a costo social hay que multiplicarlo por el costo social de cada contaminante


## 5. Definir Índices de Plantas y Bloques de Carga

In [31]:
# Índices Centrales
index_plantas = param_centrales.index.tolist()
index_plantas_nuevas = param_centrales_nuevas.index.tolist()  # demanda en MW (en la pasada tenia 12k en vez de 1200 xd)

# Índices Equipos de Abatimiento
index_equipo_mp = equipo_mp.index.tolist() # MP-1, MP-2, MP-3
index_equipo_nox = equipo_nox.index.tolist() # NOx-1, NOx-2, NOx-3
index_equipo_sox = equipo_sox.index.tolist() # SOx-1, SOx-2, SOx-3

dic_bloques = {'bloque_1': {'duracion': 1200 , 'demanda' : 10233.87729},
               'bloque_2': {'duracion': 4152 , 'demanda' : 7872.0103}, 
               'bloque_3': {'duracion': 3408 , 'demanda' : 6297.872136}}

normas_emision = {'carbon': {'MP': 0.99 , 'SOx' : 0.95, 'NOx' : 0.0},
               'cc-gnl': {'MP': 0.95 , 'SOx' : 0.0, 'NOx' : 0.9}, 
               'petroleo_diesel': {'MP': 0.95 , 'SOx' : 0.0, 'NOx' : 0.0}}

vida_util = 30 # todos los equipos tienen vida util de 30 años 

# Mostrar resumen
print('Plantillas existentes:', index_plantas[:5])
print('Plantillas nuevas:', index_plantas_nuevas[:5])

Plantillas existentes: ['planta_1', 'planta_2', 'planta_3', 'planta_4', 'planta_5']
Plantillas nuevas: ['planta_1', 'planta_2', 'planta_3', 'planta_4', 'planta_5']


In [76]:
dic = param_centrales.to_dict(orient='index')
print(dic)

{'planta_1': {'tecnologia': 'biomasa', 'ubicacion': 'itahue', 'potencia_neta_mw': 60, 'eficiencia': 0.0, 'costo_variable_nc': 0.0, 'disponibilidad': 0.9, 'costo_variable_t': 48.7, 'costo_var_comb_16_usd_mwh': 0.0, 'ED_MP_kg_ton': 0.0, 'ED_SOx_kg_ton': 0.0, 'ED_NOx_kg_ton': 0.0, 'ED_CO2_kg_ton': 0.0, 'CS_MP_USD_ton': 527, 'CS_SOx_USD_ton': 211, 'CS_NOx_USD_ton': 105, 'CS_CO2_USD_ton': 50}, 'planta_2': {'tecnologia': 'biomasa', 'ubicacion': 'charrua', 'potencia_neta_mw': 204, 'eficiencia': 0.0, 'costo_variable_nc': 0.0, 'disponibilidad': 0.9, 'costo_variable_t': 44.5, 'costo_var_comb_16_usd_mwh': 0.0, 'ED_MP_kg_ton': 0.0, 'ED_SOx_kg_ton': 0.0, 'ED_NOx_kg_ton': 0.0, 'ED_CO2_kg_ton': 0.0, 'CS_MP_USD_ton': 527, 'CS_SOx_USD_ton': 211, 'CS_NOx_USD_ton': 105, 'CS_CO2_USD_ton': 50}, 'planta_3': {'tecnologia': 'carbon', 'ubicacion': 'huasco', 'potencia_neta_mw': 575, 'eficiencia': 0.376, 'costo_variable_nc': 1.9, 'disponibilidad': 0.88, 'costo_variable_t': 0.0, 'costo_var_comb_16_usd_mwh': 22.7,

In [29]:
dic_equipo = {'MP':equipo_mp.to_dict(orient='index'), 
              'NOx':equipo_nox.to_dict(orient='index'), 
              'SOx':equipo_sox.to_dict(orient='index')}
print(dic_equipo)

#print escalonado para mostrar  lo que hay en cada diccionario
for key in dic_equipo:
    print(f"Equipos para {key}:")
    for equipo, valores in dic_equipo[key].items():
        print(f"  {equipo}: {valores}")
        


{'MP': {'MP-1': {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 116, 'Costo_variable_($/MWh)': 1.5}, 'MP-2': {'Eficiencia_(p.u.)': 0.98, 'Inversión_($/kW)': 158, 'Costo_variable_($/MWh)': 2.0}, 'MP-3': {'Eficiencia_(p.u.)': 0.99, 'Inversión_($/kW)': 274, 'Costo_variable_($/MWh)': 2.0}}, 'NOx': {'NOx-1': {'Eficiencia_(p.u.)': 0.7, 'Inversión_($/kW)': 41, 'Costo_variable_($/MWh)': 1.5}, 'NOx-2': {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 42, 'Costo_variable_($/MWh)': 2.0}, 'NOx-3': {'Eficiencia_(p.u.)': 0.95, 'Inversión_($/kW)': 63, 'Costo_variable_($/MWh)': 2.0}}, 'SOx': {'SOx-1': {'Eficiencia_(p.u.)': 0.6, 'Inversión_($/kW)': 211, 'Costo_variable_($/MWh)': 1.5}, 'SOx-2': {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 263, 'Costo_variable_($/MWh)': 2.0}, 'SOx-3': {'Eficiencia_(p.u.)': 0.95, 'Inversión_($/kW)': 316, 'Costo_variable_($/MWh)': 2.0}}}
Equipos para MP:
  MP-1: {'Eficiencia_(p.u.)': 0.9, 'Inversión_($/kW)': 116, 'Costo_variable_($/MWh)': 1.5}
  MP-2: {'Eficiencia_(p.u.)': 

## 6. Crear Modelo y Definir Conjuntos | Parámetros | Variables

In [ ]:
# Construcción del modelo Pyomo
model = pyo.ConcreteModel()

# Conjuntos
model.CENTRALES = pyo.Set(initialize=index_plantas)
model.CENTRALES_NUEVAS = pyo.Set(initialize=index_plantas_nuevas)
model.BLOQUES = pyo.Set(initialize=['bloque_1','bloque_2','bloque_3'])
# creamos el conjunto de 64 combos 
model.COMBOS_EX = pyo.Set(initialize=index_combos)  # Conjunto de plantas existentes
model.COMBOS_NUEVAS = pyo.Set(initialize=index_combos)  # Conjunto de plantas nuevas


# Parámetros en formato dict (Pyomo Any para permitir dicts anidados)
model.param_centrales = pyo.Param(model.CENTRALES, initialize=param_centrales.to_dict(orient='index'), within=pyo.Any)
model.param_centrales_nuevas = pyo.Param(model.CENTRALES_NUEVAS, initialize=param_centrales_nuevas.to_dict(orient='index'), within=pyo.Any)
model.param_bloques = pyo.Param(model.BLOQUES, initialize=dic_bloques, within=pyo.Any)
model.param_combos_ex = pyo.Param(model.I_EX, initialize=dic_combos_ex, within=pyo.Any)
model.param_combos_nuevas = pyo.Param(model.I_NUEVAS, initialize=dic_combos_nuevas, within=pyo.Any)

# Variables(todas las generaciones son en GWh y la potencia en MW)
model.generacion_ex = pyo.Var(model.CENTRALES, model.BLOQUES, within=pyo.NonNegativeReals)
model.generacion_nuevas = pyo.Var(model.CENTRALES_NUEVAS, model.BLOQUES, within=pyo.NonNegativeReals)
model.potencia_in_nuevas = pyo.Var(model.CENTRALES_NUEVAS, within=pyo.NonNegativeReals)
model.falla = pyo.Var(model.BLOQUES, within=pyo.NonNegativeReals)

model.eleccion_combo_ex = pyo.Var(model.CENTRALES, model.COMBOS_EX, within=pyo.Binary)
model.eleccion_combo_nuevas = pyo.Var(model.CENTRALES_NUEVAS, model.COMBOS_NUEVAS, within=pyo.Binary)


## 7. Definir Funciones de Restricciones

In [ ]:
# Restricciones
def fd_hidro(bloque):
    if bloque == 'bloque_1':
        return dispnibilidad_hidro[0]
    elif bloque == 'bloque_2':
        return dispnibilidad_hidro[1]
    else:
        return dispnibilidad_hidro[2]

def balance_demanda(model, bloque):
    gen_ex = sum(model.generacion_ex[planta, bloque] for planta in model.CENTRALES)
    gen_new = sum(model.generacion_nuevas[planta, bloque] for planta in model.CENTRALES_NUEVAS)
    return (gen_ex + gen_new + model.falla[bloque]) * (1000/(1+perdida)) >= model.param_bloques[bloque]['demanda'] * model.param_bloques[bloque]['duracion']
                                                    # el 1000 es para convertir de GWh a MWh

def max_gen_ex(model, planta, bloque):
    tec = model.param_centrales[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'hidro_conv', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales[planta]['disponibilidad']
        efi = model.param_centrales[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_ex[planta, bloque]*1000 <= model.param_centrales[planta]['potencia_neta_mw'] * model.param_bloques[bloque]['duracion'] * disp

def max_gen_nuevas(model, planta, bloque):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    efi = 1.0
    if tec in ['hidro', 'minihidro']:
        disp = fd_hidro(bloque)
    else:
        disp = model.param_centrales_nuevas[planta]['disponibilidad']
        efi = model.param_centrales_nuevas[planta]['eficiencia'] or 1.0
    # generacion está en GWh (multiplicamos por 1000  para hacer el cambio a MWh), potencia_neta_mw en MW, duracion en horas
    return model.generacion_nuevas[planta, bloque]*1000 <= model.potencia_in_nuevas[planta] * model.param_bloques[bloque]['duracion'] * disp

def max_capacidad_nuevas(model, planta):
    tec = model.param_centrales_nuevas[planta]['tecnologia']
    if tec in t_ernc:
        limite = model.param_centrales_nuevas[planta]['maxima_restriccion_2030_MW']
        return model.potencia_in_nuevas[planta] <= limite if limite > 0 else pyo.Constraint.Skip # Si el límite es 0, no hay restricción
    else:
        return pyo.Constraint.Skip
    
def anualidad(r, n): # r es tasa de descuento, n es vida util en años
    return r / (1 - (1 + r)**(-n))

# Adjuntar restricciones
model.demanda_constraint = pyo.Constraint(model.BLOQUES, rule=balance_demanda)
model.max_gen_constraint = pyo.Constraint(model.CENTRALES, model.BLOQUES, rule=max_gen_ex)
model.max_gen_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, model.BLOQUES, rule=max_gen_nuevas)
model.max_capacidad_nuevas_constraint = pyo.Constraint(model.CENTRALES_NUEVAS, rule=max_capacidad_nuevas)

## 9. Definir Expresiones de Costos y Función Objetivo

In [110]:
# Función objetivo (expresiones)

# Costo operación (costos variables) 2030
model.op_ex = pyo.Expression(expr=sum(model.generacion_ex[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales[planta]['costo_variable_nc']
                                + model.param_centrales[planta]['costo_variable_t']
                                + model.param_centrales[planta]['costo_var_comb_16_usd_mwh']) 
                                for planta in model.CENTRALES 
                                for bloque in model.BLOQUES)) # esto queda en USD

model.op_new = pyo.Expression(expr=sum(model.generacion_nuevas[planta, bloque] * 1000 * # pasamos a MWh
                                (model.param_centrales_nuevas[planta]['cvnc_usd_MWh'] 
                                + model.param_centrales_nuevas[planta]['linea_peaje_usd_MWh']
                                + model.param_centrales_nuevas[planta]['costo_var_comb_16_usd_mwh'])  
                                for planta in model.CENTRALES_NUEVAS 
                                for bloque in model.BLOQUES)) # esto queda en USD

# Costo inversión (anualidad) 2030
model.inv_new = pyo.Expression(expr=sum(model.potencia_in_nuevas[planta]* 
                                        anualidad(model.param_centrales_nuevas[planta]['tasa_descuento'], model.param_centrales_nuevas[planta]['vida_util_anos']) * 
                                        model.param_centrales_nuevas[planta]['inversion_usd_kW_neto']* 1000 # pasamos a MW
                                        for planta in model.CENTRALES_NUEVAS)) # esto queda en USD/año

# costo FALLAS (recordar que falla está en GWh)
model.costo_fallas = pyo.Expression(expr=sum(model.falla[bloque] * 1000 * # pasamos a MWh
                            costo_falla for bloque in model.BLOQUES))

r_df = 0.01
df_2016_2030 = 1 / (1 + r_df)**(year)

# minimizar (costo operacion + inversion + costo falla)
model.obj = pyo.Objective(expr = df_2016_2030 * (model.op_ex + model.op_new + model.inv_new + model.costo_fallas), sense = pyo.minimize)

## 10. Resolver el Modelo de Optimización

In [111]:
# Resolver el modelo
solver = pyo.SolverFactory('highs')
solver.options['mip_rel_gap'] = tolerancia
results = solver.solve(model, tee=True)
print(f"Status: {results}")

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
LP   has 131 rows; 146 cols; 314 nonzeros
Coefficient ranges:
  Matrix [1e+00, 4e+03]
  Cost   [9e+03, 4e+05]
  Bound  [0e+00, 0e+00]
  RHS    [2e+02, 3e+07]
Presolving model
63 rows, 143 cols, 243 nonzeros  0s
63 rows, 140 cols, 240 nonzeros  0s
Presolve : Reductions: rows 63(-68); columns 140(-6); elements 240(-74)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 3(4.3901e+07) 0s
         40     1.5338548176e+09 Pr: 0(0); Du: 0(1.42109e-14) 0s
Solving the original LP from the solution after postsolve
Model status        : Optimal
Simplex   iterations: 40
Objective value     :  1.5338548176e+09
P-D objective error :  1.5543751357e-16
HiGHS run time      :          0.00
Status: 
Problem: 
- Lower bound: 1533854817.584971
  Upper bound: 1533854817.584971
  Number of objectives: 1
  Number 

## 11. Análisis de Resultados por Tipo de Tecnología

In [112]:
# ver resultados por tipo de central
for tec in t_centrales:
    gen_tec = sum(model.generacion_ex[planta, bloque].value 
                  for planta in model.CENTRALES if model.param_centrales[planta]['tecnologia'] == tec
                  for bloque in model.BLOQUES)
    gen_tec_nuevas = sum(model.generacion_nuevas[planta, bloque].value 
                  for planta in model.CENTRALES_NUEVAS if model.param_centrales_nuevas[planta]['tecnologia'] == tec
                  for bloque in model.BLOQUES)
    gen_tec += gen_tec_nuevas
    print(f'Generación total de tecnología {tec}: {gen_tec} GWh')

Generación total de tecnología biomasa: 1271.6352000000002 GWh
Generación total de tecnología carbon: 34581.53984929151 GWh
Generación total de tecnología cc-gnl: 815.796 GWh
Generación total de tecnología petroleo_diesel: 849.5926435200009 GWh
Generación total de tecnología hidro: 30708.4795704 GWh
Generación total de tecnología minihidro: 0.0 GWh
Generación total de tecnología eolica: 420.48 GWh
Generación total de tecnología solar: 438.0 GWh
Generación total de tecnología geotermia: 0.0 GWh


## 12. Análisis de Generación Total del Sistema y Balance de Carga

In [113]:
#sumatoria todas las generaciones
total = 0
for bloque in model.BLOQUES:
    gen_ex_bloque = sum(model.generacion_ex[planta, bloque].value for planta in model.CENTRALES)
    gen_new_bloque = sum(model.generacion_nuevas[planta, bloque].value for planta in model.CENTRALES_NUEVAS)
    falla_bloque = model.falla[bloque].value
    print(f'Generación total en {bloque}: {gen_ex_bloque+gen_new_bloque} GWh, Falla: {falla_bloque} GWh')
    total += gen_ex_bloque + gen_new_bloque + falla_bloque

print(f'Generación total en el sistema (incluyendo fallas): {total} GWh')

Generación total en bloque_1: 12771.87885792 GWh, Falla: 0.0 GWh
Generación total en bloque_2: 33991.970236224 GWh, Falla: 0.0 GWh
Generación total en bloque_3: 22321.67416906752 GWh, Falla: 0.0 GWh
Generación total en el sistema (incluyendo fallas): 69085.52326321151 GWh
